# Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle

# Usado en ronda 1, 2 de relacionar animes.
import re
# Usado en ronda 3 de relacionar.
from difflib import SequenceMatcher

# Combinar data de Web-Scraping y Api en archivos unicos.

Las series de archivos que se combinaran son los siguientes

Con i entre 1 y 20:

1. listas_usuarios_i.pickle
2. detalles_anime_i.pickle
3. tags_anime_i.pickle

Usuarios_i.json no se combinaran, pues estos solo sirven para generar listas_usuarios_i.pickle

# Relacionar MyAnimeList con AnimePlanet

El proposito de este proceso es combinar las id's de MyAnimeList con los nombres de AnimePlanet.

El proceso sigue 3 rondas.

1. Busca todos los titulos en los detalles del anime de MyAnimeList, luego busca si encuentra alguno de ellos en Animeplanet

2. Compara todos los titulos de MyAnimeList con todos los titulos de Animeplanet.

3. Si eso falla, se hara a fuerza bruta encontrar el titulo mas similar, este proceso puede tardar mas de una hora. Y si el titulo es muy disimilar, entonces no se considerara.

4. Al final, el restante deberia ser suficientemente chico como para revisar a mano.

5. Despues de eso, deberia ser necesario revisar que no hayan quedado duplicados, en cuyo caso hay que arreglarlos

In [ ]:
all_anime_df = pd.DataFrame()
# tags = pd.read_csv(os.path.join("data", "tags_anime_1.pickle"))

for i in range(1, 21):
    tags = pd.read_csv(os.path.join("data", f"tags_anime_{i}.pickle"))
    all_anime_df = pd.concat([all_anime_df, tags])
all_anime_df.reset_index(drop=True, inplace=True)

# Convertir tags a una lista
all_anime_df["Tags"] = all_anime_df["Tags"].apply(lambda x: [x.strip("'") for x in x.lstrip("[").rstrip("]").split(", ")])
all_anime_df.head(3)

,Anime,URL,Tags,Alts
0,Attack on Titan The Final Season: The Final Ch...,https://www.anime-planet.com/anime/attack-on-t...,"[Action, Drama, Fantasy, Shounen, Dark Fantasy...",\r\nAlt titles: Attack on Titan The Final Seas...
1,Fullmetal Alchemist: Brotherhood,https://www.anime-planet.com/anime/fullmetal-a...,"[Action, Adventure, Drama, Fantasy, Mystery, S...",\r\nAlt title: Hagane no Renkinjutsushi: Full ...
2,Fruits Basket the Final Season,https://www.anime-planet.com/anime/fruits-bask...,"[Drama, Fantasy, Romance, Shoujo, Animal Trans...",\r\nAlt title: Fruits Basket the Final\r\n


In [ ]:
%cd ..
%cd demo Mativ
all_anime_df = pd.read_csv(os.path.join("data", "tags_anime.csv"))
%cd ..
%cd demo Victor

c:\repos\datos\Proyecto\Proyecto-Grupo-12
c:\repos\datos\Proyecto\Proyecto-Grupo-12\demo Mativ
c:\repos\datos\Proyecto\Proyecto-Grupo-12
c:\repos\datos\Proyecto\Proyecto-Grupo-12\demo Victor


In [ ]:
detalles_anime = {}
for i in range(1, 21):
    path = os.path.join("data", f"detalles_anime_{i}.pickle")
    with open(path, "rb") as archivo:
        detalles_anime = detalles_anime | pickle.load(archivo)

## Ronda 1

In [ ]:
dummy_df = all_anime_df.copy()
dummy_df["Anime"] = dummy_df["Anime"].str.lower()
dummy_df["Anime"] = dummy_df["Anime"].str.replace(r"[^a-zA-Z0-9]", "", regex=True)

pd.options.mode.chained_assignment = None  # default='warn'

titulos_1 = pd.DataFrame()
not_found = {}
for anime_id, value in detalles_anime.items():
    details = value["detalles"]
    lista_titulos = [details["title"]] + (details["alternative_titles"]["synonyms"]) + [(details["alternative_titles"]["en"])]
    for titulo in lista_titulos:
        titulo_plano = re.sub("[^a-zA-Z0-9]", "", titulo.lower())
        if not titulo_plano:
            continue
        mini = all_anime_df[dummy_df["Anime"] == titulo_plano]
        if not mini.empty:
            break
    else: # No se encontro el anime
        not_found[anime_id] = lista_titulos
        continue
    mini["MAL_ID"] = anime_id
    titulos_1 = pd.concat([titulos_1, mini])
    
path = os.path.join("research", "titulos_1.csv")
titulos_1.to_csv(path, index=False)

titulos_1

,Anime,URL,MAL_ID
112,Cowboy Bebop,https://www.anime-planet.com/anime/cowboy-bebop,1
13925,Puzzle & Dragons X,https://www.anime-planet.com/anime/puzzle-and-...,32772
353,Cowboy Bebop: The Movie,https://www.anime-planet.com/anime/cowboy-bebo...,5
422,Trigun,https://www.anime-planet.com/anime/trigun,6
4663,Witch Hunter Robin,https://www.anime-planet.com/anime/witch-hunte...,7
...,...,...,...
3827,Girls und Panzer der Film: Arisu War!,https://www.anime-planet.com/anime/girls-und-p...,32740
959,Dragon Ball Super,https://www.anime-planet.com/anime/dragon-ball...,30694
13023,Pop in Q,https://www.anime-planet.com/anime/pop-in-q,30695
2427,The Guardian Brothers,https://www.anime-planet.com/anime/the-guardia...,32751


## Ronda 2

In [ ]:
some_anime_df = all_anime_df[~all_anime_df.index.isin(titulos_1.index)]

dummy_df_2 = some_anime_df.copy()
dummy_df_2["Anime"] = dummy_df_2["Anime"].str.lower()
dummy_df_2["Anime"] = dummy_df_2["Anime"].str.replace(r"[^a-zA-Z0-9]", "", regex=True)

dummy_df_2["Alts"].fillna("", inplace=True)
dummy_df_2["Alts"] = all_anime_df["Alts"].apply(lambda x: x.strip("\r\n"))
dummy_df_2["Alts"] = dummy_df_2["Alts"].apply(lambda x: x.replace("Alt title: ", "").replace("Alt titles: ", "").split(", "))

not_found_2 = {}
titulos_2 = pd.DataFrame()

for anime_id, value in not_found.items():
    details = value["detalles"]
    lista_titulos = [details["title"]] + (details["alternative_titles"]["synonyms"]) + [(details["alternative_titles"]["en"])]

    for titulo in lista_titulos:
        # Se busca el titulo mas parecido
        titulo_plano = re.sub("[^a-zA-Z0-9]", "", titulo.lower())
        if not titulo_plano:
            continue
        mini = some_anime_df[dummy_df_2["Alts"].apply(lambda x: titulo_plano in x)]
        if not mini.empty:
            break
    else: # No se encontro el anime
        not_found[anime_id] = lista_titulos
        continue
    mini["MAL_ID"] = anime_id
    titulos_2 = pd.concat([titulos_2, mini])

path = os.path.join("research", "titulos_2.csv")
titulos_2.to_csv(path, index=False)

titulos_2

2274

## Ronda 3

In [ ]:
# dataframe de all_anime_df sin titulos
last_anime_df = all_anime_df[~((all_anime_df.index.isin(titulos_1.index)) | (all_anime_df.index.isin(titulos_2.index)))]

dummy_df_3 = last_anime_df.copy()
dummy_df_3["Anime"] = dummy_df_3["Anime"].str.lower()

not_found_3 = {}

titulos_3 = pd.DataFrame()

for anime_id, value in not_found.items():
    details = value["detalles"]
    lista_titulos = [details["title"]] + (details["alternative_titles"]["synonyms"]) + [(details["alternative_titles"]["en"])]

    max_score = 0
    max_score_id = None
    for titulo in lista_titulos:
        # Se busca el titulo mas parecido
        ratios = dummy_df_3["Anime"].apply(lambda x: SequenceMatcher(None, x, titulo.lower()).ratio())

        score_id = ratios.idxmax()
        score = ratios[score_id]
    
        if score > max_score:
            max_score = score
            max_score_id = score_id
    if max_score < 0.7:
        print(anime_id, lista_titulos, max_score)
        not_found_3[anime_id] = lista_titulos
        continue
    mini = last_anime_df.loc[[max_score_id]]
    mini["MAL_ID"] = anime_id
    titulos_2 = pd.concat([titulos_2, mini])
path = os.path.join("research", "titulos_3.csv")
titulos_3.to_csv(path, index=False)

# Crear dataframe de ratings de usuarios